# Building a Chatbot with RAG

In [ ]:
!pip install -qU \
    langchain==0.0.354 \
    openai==1.6.1 \
    datasets==2.10.1 \
    pinecone-client==3.0.0 \
    tiktoken==0.5.2

#Building a Chatbot (no RAG)
We will be relying heavily on the LangChain library to bring together the different components needed for our chatbot. To begin, we'll create a simple chatbot without any retrieval augmentation. We do this by initializing a ChatOpenAI object. For this we do need an OpenAI API key.

In [ ]:
! pip install -q --upgrade google-generativeai langchain-google-genai python-dotenv

In [ ]:
!pip install -q langchain

In [ ]:
import pathlib  # For working with file paths
import textwrap  # For wrapping text
from IPython.display import display, Markdown  # For displaying output in Jupyter Notebook


# Function to convert text to Markdown format
def to_markdown(text):
    # Replace bullet points with Markdown list items
    text = text.replace('•', '  *')
    # Indent the text with '>' and return it as Markdown
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


In [ ]:
# importing the libraries.
import google.generativeai as genai
import os
from dotenv import load_dotenv, find_dotenv
import getpass

os.environ["GOOGLE_API_KEY"] = getpass.getpass("Paste your Google API key here and hit enter:")   #user input


# loading the API key from .env and authenticating to Gemini.
load_dotenv(find_dotenv(), override=True)
os.environ.get('GOOGLE_API_KEY')

# configuring the API key.
genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

Paste your Google API key here and hit enter:··········


In [ ]:
# generation_config = {
#         "temperature": 0.0,
#         # "top_p": 1,
#         # "top_k": 1,
#         "max_output_tokens": 2048,
# }

# model = genai.GenerativeModel(generation_config=generation_config, model_name='gemini-pro')
prompt = 'give me guidelines on how to structure my timetable between school, work, personal development'

model = genai.GenerativeModel('gemini-pro')
response = model.generate_content(prompt)
# print(response.text)

Gemini can generate multiple possible responses for a single prompt. These possible responses are called `candidates`, and you can review them to select the most suitable one as the response.

View the response candidates with `GenerateContentResponse.candidates:`

In [ ]:
response.candidates

[content {
  parts {
    text: "**Prioritize Your Commitments:**\n\n1. **School:** Make school your primary focus. Identify essential classes and assignments and plan study sessions accordingly.\n\n2. **Work:** Calculate the hours you need to work to meet your financial obligations and goals.\n\n3. **Personal Development:** Include activities that enhance your skills, knowledge, and well-being. This could include exercise, hobbies, or learning new skills.\n\n**Create a Timetable:**\n\n1. **Flexibility:** Design a timetable that adapts to your daily and weekly fluctuations. Allow for unexpected events and breaks.\n\n2. **Color-Coding:** Use different colors to distinguish between school, work, and personal development activities.\n\n3. **Breaks:** Incorporate short breaks throughout the day to avoid burnout and maintain productivity.\n\n4. **Realism:** Be realistic about your time commitment to each activity. Avoid over-scheduling yourself.\n\n5. **Daily Schedule:** Plan out each day in

In [ ]:
for chunk in response:
  print(chunk.text)
  print("_"*80)

**Prioritize Your Commitments:**

1. **School:** Make school your primary focus. Identify essential classes and assignments and plan study sessions accordingly.

2. **Work:** Calculate the hours you need to work to meet your financial obligations and goals.

3. **Personal Development:** Include activities that enhance your skills, knowledge, and well-being. This could include exercise, hobbies, or learning new skills.

**Create a Timetable:**

1. **Flexibility:** Design a timetable that adapts to your daily and weekly fluctuations. Allow for unexpected events and breaks.

2. **Color-Coding:** Use different colors to distinguish between school, work, and personal development activities.

3. **Breaks:** Incorporate short breaks throughout the day to avoid burnout and maintain productivity.

4. **Realism:** Be realistic about your time commitment to each activity. Avoid over-scheduling yourself.

5. **Daily Schedule:** Plan out each day in specific time slots, including waking up, commuti

In [ ]:
model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])
chat

In [ ]:
response = chat.send_message("In one sentence, explain how a computer works to a young child.")
to_markdown(response.text)

> A computer is like a magical helper that can store and show you things, play games, and help you learn.

In [ ]:
chat.history

[parts {
   text: "In one sentence, explain how a computer works to a young child."
 }
 role: "user",
 parts {
   text: "A computer is like a magical helper that can store and show you things, play games, and help you learn."
 }
 role: "model"]

In [ ]:
response = chat.send_message("Okay, how about a more detailed explanation to a high schooler?", stream=True)

for chunk in response:
  print(chunk.text)for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
  print("_"*80)

At its core, a computer is an electronic device that can be programmed to carry
________________________________________________________________________________
 out a set of instructions. It consists of hardware, which are the physical components, and software, which are the instructions that tell the hardware what to do.
________________________________________________________________________________
 When you give a computer a command, the software translates it into a series of binary digits (0s and 1s) that the hardware can understand. The hardware then carries out the instructions, processes the data, and displays the results on the screen or through other output devices. Computers can perform complex calculations, store
________________________________________________________________________________
 and retrieve vast amounts of information, and communicate with other computers over networks.

In summary, a computer is a programmable electronic device that can process informatio

"streaming" means that the response is being delivered in chunks or segments rather than as a single block of text.

In [ ]:
for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))

> **user**: In one sentence, explain how a computer works to a young child.

> **model**: A computer is like a magical helper that can store and show you things, play games, and help you learn.

> **user**: Okay, how about a more detailed explanation to a high schooler?

> **model**: At its core, a computer is an electronic device that can be programmed to carry out a set of instructions. It consists of hardware, which are the physical components, and software, which are the instructions that tell the hardware what to do. When you give a computer a command, the software translates it into a series of binary digits (0s and 1s) that the hardware can understand. The hardware then carries out the instructions, processes the data, and displays the results on the screen or through other output devices. Computers can perform complex calculations, store and retrieve vast amounts of information, and communicate with other computers over networks.
> 
> In summary, a computer is a programmable electronic device that can process information, store data, and communicate with other computers.

In [ ]:
result = genai.embed_content(
    model="models/embedding-001",
    content="What is the meaning of life?",
    task_type="retrieval_document",
    title="Embedding of single string")

# 1 input > 1 vector output
print(str(result['embedding'])[:50], '... TRIMMED]')

[-0.003216741, -0.013358698, -0.017649598, -0.0091 ... TRIMMED]


In [ ]:
result = genai.embed_content(
    model="models/embedding-001",
    content=[
      'What is the meaning of life?',
      'How much wood would a woodchuck chuck?',
      'How does the brain work?'],
    task_type="retrieval_document",
    title="Embedding of list of strings")

# A list of inputs > A list of vectors output
for v in result['embedding']:
  print(str(v)[:50], '... TRIMMED ...')

[0.0040260437, 0.004124458, -0.014209415, -0.00183 ... TRIMMED ...
[-0.004049845, -0.0075574904, -0.0073463684, -0.03 ... TRIMMED ...
[0.025310587, -0.0080734305, -0.029902633, 0.01160 ... TRIMMED ...


In [ ]:
response.candidates[0].content

parts {
  text: "At its core, a computer is an electronic device that can be programmed to carry out a set of instructions. It consists of hardware, which are the physical components, and software, which are the instructions that tell the hardware what to do. When you give a computer a command, the software translates it into a series of binary digits (0s and 1s) that the hardware can understand. The hardware then carries out the instructions, processes the data, and displays the results on the screen or through other output devices. Computers can perform complex calculations, store and retrieve vast amounts of information, and communicate with other computers over networks.\n\nIn summary, a computer is a programmable electronic device that can process information, store data, and communicate with other computers."
}
role: "model"

In [ ]:
result = genai.embed_content(
    model = 'models/embedding-001',
    content = response.candidates[0].content)

# 1 input > 1 vector output
print(str(result['embedding'])[:50], '... TRIMMED ...')

[-0.012603716, -0.046497587, 0.0021071662, 0.02191 ... TRIMMED ...


In [ ]:
chat.history

[parts {
   text: "In one sentence, explain how a computer works to a young child."
 }
 role: "user",
 parts {
   text: "A computer is like a magical helper that can store and show you things, play games, and help you learn."
 }
 role: "model",
 parts {
   text: "Okay, how about a more detailed explanation to a high schooler?"
 }
 role: "user",
 parts {
   text: "At its core, a computer is an electronic device that can be programmed to carry out a set of instructions. It consists of hardware, which are the physical components, and software, which are the instructions that tell the hardware what to do. When you give a computer a command, the software translates it into a series of binary digits (0s and 1s) that the hardware can understand. The hardware then carries out the instructions, processes the data, and displays the results on the screen or through other output devices. Computers can perform complex calculations, store and retrieve vast amounts of information, and communicate wit

### Multi-turn conversations

While the `genai.ChatSession` class shown earlier can handle many use cases, it does make some assumptions. If your use case doesn't fit into this chat implementation it's good to remember that `genai.ChatSession` is just a wrapper around <a href="https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content"><code>GenerativeModel.generate_content</code></a>. In addition to single requests, it can handle multi-turn conversations.

The individual messages are `glm.Content` objects or compatible dictionaries, as seen in previous sections. As a dictionary, the message requires `role` and `parts` keys. The `role` in a conversation can either be the `user`, which provides the prompts, or `model`, which provides the responses.

Pass a list of `glm.Content` objects and it will be treated as multi-turn chat:

In [ ]:
model = genai.GenerativeModel('gemini-pro')

messages = [
    {'role':'model',
    # 'role':'you are a helpful, patient and intelligent customer support agent', (not supportd for gemini-pro)
    'role':'user',
    'parts': ["Briefly explain how to buy your $400 product."]}
]
response = model.generate_content(messages)

to_markdown(response.text)

> 1. Go to our official website: [Website URL]
> 
> 
> 2. Browse our product catalog and select the $400 product you want to purchase.
> 
> 
> 3. Click on the "Add to Cart" button to add the product to your shopping cart.
> 
> 
> 4. Click on the "Checkout" button to proceed to the checkout page.
> 
> 
> 5. Enter your shipping and billing information.
> 
> 
> 6. Select your preferred payment method and enter your payment information.
> 
> 
> 7. Review your order summary and click on the "Confirm Order" button to complete your purchase.
> 
> 
> 8. You will receive an email with your order confirmation and tracking information.
> 
> 
> 9. Your order will be processed and shipped within [Shipping Timeframe] business days.
> 
> 
> 10. Once your order is delivered, inspect the product and contact our customer support team if you have any questions or concerns.

To continue the conversation, add the response and another message.

Note: For multi-turn conversations, you need to send the whole conversation history with each request. The API is stateless.

In [ ]:
messages.append({'role':'model',
                 'parts':[response.text]})

messages.append({'role':'user',
                 'parts':["Okay, how about a more detailed explanation to a high school student?"]})

response = model.generate_content(messages)

to_markdown(response.text)

> 1. **Go to the Website:**
> 
>    - Open a web browser on your computer or mobile device and type in the URL of our official website: [Website URL].
>    - Make sure you are on the official website to ensure a secure and legitimate purchase.
> 
> 
> 2. **Find the Product:**
> 
>    - Browse through the product catalog on the website. You can use the search bar or navigate through the different categories to find the specific $400 product you want to buy.
> 
> 
> 3. **Add to Cart:**
> 
>    - Once you find the product, click on the "Add to Cart" button. This will add the product to your shopping cart.
>    - You can continue browsing and adding more products to your cart if you wish.
> 
> 
> 4. **Proceed to Checkout:**
> 
>    - When you are ready to purchase the items in your cart, click on the "Checkout" button. This will take you to the checkout page.
> 
> 
> 5. **Enter Shipping and Billing Information:**
> 
>    - On the checkout page, enter your shipping address and billing information. Make sure the information is accurate to ensure proper delivery and billing.
> 
> 
> 6. **Choose Payment Method:**
> 
>    - Select your preferred payment method from the available options. We typically offer various payment methods such as credit/debit cards, PayPal, or other secure payment gateways.
> 
> 
> 7. **Review and Confirm Order:**
> 
>    - Review your order summary carefully. Ensure that the products, quantities, shipping address, and payment information are all correct.
>    - Once you are satisfied with your order, click on the "Confirm Order" or "Place Order" button to complete your purchase.
> 
> 
> 8. **Order Confirmation:**
> 
>    - You will receive an email with your order confirmation and a unique order number. This email will also contain tracking information once your order is shipped.
> 
> 
> 9. **Processing and Shipping:**
> 
>    - Your order will be processed and shipped within the specified shipping timeframe, typically within a few business days. You can track the status of your order using the tracking information provided in the confirmation email.
> 
> 
> 10. **Delivery and Inspection:**
> 
>     - Once your order is delivered, inspect the product to ensure that it is in good condition and matches your expectations.
>     - If you have any questions or concerns about the product, contact our customer support team for assistance.

In [ ]:
model = genai.GenerativeModel('gemini-pro')
response = model.generate_content(
    'how do i make my first million dollars from forex and data science?',
    generation_config=genai.types.GenerationConfig(
        # Only one candidate for now.
        candidate_count=1,
        # stop_sequences=['x'],
        # max_output_tokens=20,
        temperature=0.0)
)

In [ ]:
text = response.text

# if response.candidates[0].finish_reason.name == "MAX_TOKENS":
#     text += '...'

to_markdown(text)

> 1. **Learn the Basics of Forex Trading:**
>    - Understand the concepts of currency pairs, pips, leverage, and risk management.
>    - Familiarize yourself with different trading strategies and technical indicators.
>    - Practice trading on a demo account before using real money.
> 
> 2. **Master Data Science Techniques:**
>    - Learn programming languages like Python and R, which are widely used in data science.
>    - Study statistical methods, machine learning algorithms, and data visualization techniques.
>    - Gain experience working with large datasets and building predictive models.
> 
> 3. **Combine Forex Trading and Data Science:**
>    - Use data science techniques to analyze historical forex data and identify trading opportunities.
>    - Develop automated trading strategies based on machine learning algorithms.
>    - Backtest your strategies on historical data to assess their performance.
> 
> 4. **Choose a Reliable Forex Broker:**
>    - Select a regulated and reputable forex broker that offers competitive trading conditions.
>    - Consider factors such as spreads, commissions, leverage, and customer support.
> 
> 5. **Start with a Small Account:**
>    - Begin trading with a small amount of money to minimize your risk.
>    - Gradually increase your trading size as you gain experience and confidence.
> 
> 6. **Follow a Consistent Trading Plan:**
>    - Create a trading plan that outlines your trading strategy, risk management rules, and entry and exit criteria.
>    - Stick to your plan and avoid making impulsive trades.
> 
> 7. **Manage Your Risk:**
>    - Use stop-loss orders to limit your potential losses on each trade.
>    - Diversify your portfolio by trading multiple currency pairs.
>    - Avoid overtrading and maintain a healthy risk-to-reward ratio.
> 
> 8. **Stay Updated with Market News and Analysis:**
>    - Follow economic news and events that can impact currency prices.
>    - Read market analysis reports and commentaries from reputable sources.
>    - Stay informed about geopolitical developments and central bank policies.
> 
> 9. **Be Patient and Persistent:**
>    - Forex trading is a long-term game that requires patience and persistence.
>    - Don't expect to make millions overnight.
>    - Focus on learning, improving your skills, and managing your risk.
> 
> 10. **Seek Professional Guidance:**
>     - Consider working with a mentor or coach who has experience in forex trading and data science.
>     - Attend workshops, seminars, and online courses to enhance your knowledge and skills.

In [ ]:
  model = genai.GenerativeModel('gemini-pro')
  chat = model.start_chat(history=[])

  response = chat.send_message(
      "you\'re a helpful, patient amd intelligent customer support bot")
  print(response.text)

  response = chat.send_message(
      "how do i make returns and complaints for the product i ordered?")
  print(response.text)

As a helpful, patient, and intelligent customer support bot, I am designed to assist you with any inquiries or issues you may have. Here are some of my key characteristics:

**Helpful:**

* I am programmed with a vast knowledge base that covers a wide range of topics and products.
* I can understand and respond to your questions and requests in a clear and informative manner.
* I am constantly learning and updating my knowledge to provide you with the most accurate and up-to-date information.

**Patient:**

* I understand that sometimes customers may have complex issues or inquiries that require a detailed explanation.
* I am always willing to take the time to thoroughly address your concerns and provide you with a satisfactory resolution.
* I am patient and understanding, even if you are frustrated or upset.

**Intelligent:**

* I am equipped with advanced artificial intelligence (AI) capabilities that allow me to analyze and interpret your queries quickly and accurately.
* I can unde